In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import random

import nest_asyncio
nest_asyncio.apply()
import tensorflow_federated as tff

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
#from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model


2022-06-08 06:54:49.811436: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-08 06:54:49.811481: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
BATCH_SIZE = 32
NUM_CLIENTS = 10
BATCH_SIZE = 1

In [3]:
df = pd.read_csv("../datasets/TON_IoT-Datasets/Train_Test_datasets/Train_Test_Network_dataset/Train_Test_Network.csv")
df.head()
#reduced_df = df[["src_port", "dst_port", "label"]]
#df = reduced_df

#columns = range(3,8)
#df = dataframe.iloc[:, columns]

,ts,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,...,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,1554198358,3.122.49.24,1883,192.168.1.152,52976,tcp,-,80549.530260,1762852,41933215,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
1,1554198358,192.168.1.79,47260,192.168.1.255,15600,udp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
2,1554198359,192.168.1.152,1880,192.168.1.152,51782,tcp,-,0.000000,0,0,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
3,1554198359,192.168.1.152,34296,192.168.1.152,10502,tcp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
4,1554198362,192.168.1.152,46608,192.168.1.190,53,udp,dns,0.000549,0,298,...,0,0,-,-,-,bad_UDP_checksum,-,F,0,normal


In [4]:

#IID or NOIID? Random split
client_id = np.random.choice(range(NUM_CLIENTS), size=len(df), replace=True)
#df['client_id'] = client_id
df.head()

,ts,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,...,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,1554198358,3.122.49.24,1883,192.168.1.152,52976,tcp,-,80549.530260,1762852,41933215,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
1,1554198358,192.168.1.79,47260,192.168.1.255,15600,udp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
2,1554198359,192.168.1.152,1880,192.168.1.152,51782,tcp,-,0.000000,0,0,...,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
3,1554198359,192.168.1.152,34296,192.168.1.152,10502,tcp,-,0.000000,0,0,...,0,0,-,-,-,-,-,-,0,normal
4,1554198362,192.168.1.152,46608,192.168.1.190,53,udp,dns,0.000549,0,298,...,0,0,-,-,-,bad_UDP_checksum,-,F,0,normal


In [5]:
netw_features = df.copy()
netw_features.pop('type')
#netw_labels = netw_features.pop('label')


0             normal
1             normal
2             normal
3             normal
4             normal
             ...    
461038           xss
461039    ransomware
461040      backdoor
461041          mitm
461042          mitm
Name: type, Length: 461043, dtype: object

In [6]:
#netw_features_dict = {name: np.array(value) 
#                         for name, value in netw_features.items()}


In [7]:
#Create a dict where keys are client ids format for tff.simulation.datasets.TestClientData
df_cl_dict = {}
for id in range(NUM_CLIENTS):
    rslt_df = netw_features.loc[client_id == id]
    tmp_netw_features_dict = {name: np.array(value) 
                         for name, value in rslt_df.items()}
    df_cl_dict[str(id)]=tmp_netw_features_dict.copy()

In [8]:
#for name, value in df_cl_dict.items():
#  print(f"{name:19s}: {value}")
#  break

In [9]:
#netw_ds = tf.data.Dataset.from_tensor_slices((netw_features_dict, netw_labels))
#netw_ds = tf.data.Dataset.from_tensor_slices(netw_features_dict)
netw_fd_ds = tff.simulation.datasets.TestClientData(df_cl_dict)



2022-06-08 06:54:56.072191: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-08 06:54:56.072221: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-08 06:54:56.072256: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (debian): /proc/driver/nvidia/version does not exist
2022-06-08 06:54:56.072479: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
# for example in netw_fd_ds:
#   for name, value in example[0].items():
#     print(f"{name:19s}: {value}")
#   break
# # SHUFFLE DATA IN BATCH SIZES
# netw_batches = netw_ds.shuffle(len(netw_ds)).batch(BATCH_SIZE)

## Split in train and test TFF